In [5]:
from typing import Tuple, Mapping

import numpy as np
from matplotlib import pyplot as plt
import sklearn as skl
from sklearn import datasets
from matplotlib.animation import FuncAnimation
from IPython import display
from numpy.random import permutation


In [118]:
def stochastic_sample(xs, ys):
    """
        stochastic_sample: sample with replacement one x and one y
        xs: all point on the plane
        ys: all response on the plane
        
        return the one randomly selected x and y point
    """
    perm = permutation(len(xs))
    x = xs[perm[0]]
    y = ys[perm[0]]
    return x, y

class lenDataloader():
    '''бесмысленый класс для создания случайного бача из пар точек x,y созданный пока я забыл что эти точки и есть оптимизационные параметры'''
    # инициатия двухмерного датасета
    def __init__(self,x,y,batch_size: int = 1,shuffle: bool = True,drop_last: bool = False) -> None:
        self.X=x
        self.Y=y
        self.batch_size=batch_size
        self.drop_last=drop_last
        self.shuffle=shuffle
        self.batchindex=0
        self.endbatchindex=0
        self.X1=[]
        self.Y1=[]
    # отображение длины датасеета 
    def len(self) -> int:
        return len(self.X)
    # перемешивания индексов датасета если включено перемешивание иначе копируем
    def permutation(self):
        if (self.shuffle==True):
            perm = permutation(self.len)
            self.X1=[self.X[ind] for ind in perm]
            self.Y1=[self.Y[ind] for ind in perm]
        else:
            self.X1=self.X
            self.Y1=self.Y
    def ReturnPermDataset(self,indxstart:int,indxstop=None,indxstep=1):
        if indxstop==None:
            return self.X1[indxstart],self.Y1[indxstart]
        else:
            return [self.X1[indxstart:indxstop:indxstep],self.Y1[indxstart:indxstop:indxstep]]
    
    def returnBatchSet(self):
        if (self.batchindex==0):
            # инициализация доп сета для перемешивания 
            self.permutation()
            self.endbatchindex=(self.len() // self.batch_size)-1
            # если датасет не делиться на ровные батчи отбрасываем последний по флагу
            if ((self.drop_last==True) and (self.len() % self.batch_size !=0)):
                self.endbatchindex-=1
        
        if ((self.drop_last==False) and (self.len() % self.batch_size !=0)and (self.batchindex+1==self.endbatchindex)):
            batch[self.ReturnPermDataset(self.batchindex*self.batch_size, self.len())]
        else:
            batch=[self.ReturnPermDataset(self.batchindex*self.batch_size, self.batchindex*self.batch_size+self.batch_size)]
        self.batchindex+=1
        
        return batch[0],batch[1]
    def endEpoch(self):
        self.batchindex=0

In [129]:
#функция 1
def Himmelblau(x: np.float32, y: np.float32) -> np.float64:
    '''
    Функция Химмельблау
    
    Args:
        x(np.ndarray): Аргумент 1
        y(np.ndarray): Аргумент 2
        
    Returns:
        np.float64: Результат функции
    '''
    return (np.square(np.square(x)+y-11)+np.square(x+np.square(y)-7));

#градиент 1
def Himmelblau_Grad(x: np.ndarray, y: np.ndarray) -> np.float64:
    return np.array([(4*x*(np.square(x)+y-11)+2*(x+np.square(y)-7)), (2*(np.square(x)+y-11)+4*y*(x+np.square(y)-7))]);

Himmelblau(2, 3), Himmelblau_Grad(2, 3)

(32, array([-24,  40]))

In [48]:
#набор точек
x, y = skl.datasets.make_blobs(n_samples=2, centers=None, n_features=1, random_state=0)
y=np.reshape(y,x.shape)

In [82]:
from symbol import yield_stmt


def mu_func(x: np.float32, y: np.float32) -> np.float64:
    '''
    Функция Химмельблау
    
    Args:
        x(np.ndarray): Аргумент 1
        y(np.ndarray): Аргумент 2
        
    Returns:
        np.float64: Результат функции
    '''
    return x**2+y**2;

#градиент 1
def mu_func_Grad(x: np.ndarray, y: np.ndarray) -> np.float64:
    return np.array([2*x, 2*y]);


In [131]:
a=np.array([1.0,2.,3.,4.,1.])
b=np.array([1.,2.,3.,4.,1.])

In [132]:
mu_func_Grad(a,b)

array([[2., 4., 6., 8., 2.],
       [2., 4., 6., 8., 2.]])

In [139]:
my_GD_LRS_nest(Himmelblau,Himmelblau_Grad,a,b,lr=0.1,lre=20)

(array([2.9998581 , 2.99988328, 2.9999038 , 3.00003285, 2.9998581 ]),
 array([2.00034244, 2.00028171, 2.00023218, 1.9999207 , 2.00034244]),
 array([1.76693815e-06, 1.19572409e-06, 8.12212956e-07, 9.47322856e-08,
        1.76693815e-06]))

In [100]:
#GDF
def my_GD(f: Mapping, df: Mapping, x: np.ndarray, y: np.ndarray, lr: float = 0.01,
          Epoch: int = 100) -> Tuple [np.ndarray, np.ndarray, np.float32]:
    '''Моя простейшая реализация градиентного спуска.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка 1
        y0 (np.ndarray): Стартовая точка 2
        lr (float): Скорость обучения. Default=0,01.
        T (int): Количество итераций.
    
    Returns:
        Tuple [np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
    
    '''

    for i in range(Epoch):
        print(x,y)
        n=df(x, y)
        print(n)
        x = x - lr*n[0]
        y = y - lr*n[1]
        
    return x, y, f(x, y)
    

In [89]:
my_GD(Himmelblau,Himmelblau_Grad,[0],[0])

(array([3., 3., 3., 3.]),
 array([2., 2., 2., 2.]),
 array([2.23017289e-24, 2.23017289e-24, 2.23017289e-24, 2.23017289e-24]))

In [103]:
def my_GD_LRS(f: Mapping, df: Mapping, x: np.ndarray, y: np.ndarray, lr: float = 0.01,
          Epoch: int = 100, lre: int = 7, g: float = 0.1) -> Tuple [np.ndarray, np.ndarray, np.float32]:
    '''
    Моя простейшая реализация градиентного спуска + Learning Rate Schedule.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка 1
        y0 (np.ndarray): Стартовая точка 2
        lr (float): Скорость обучения. Default=0,01.
        T (int): Количество итераций.
        lre (int): Через сколько эпох мы уменьшим lr. Default = 7.
        g (float): Коэфициент уменьшения lr [0,1). Default = 0.1
    
    Returns:
        Tuple [np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
    
    '''

    for i in range(Epoch):
        if ((i%lre)==0):
            lr *= g
        n=df(x, y)
        x = x - lr*n[0]
        y = y - lr*n[1]
        
    return x, y, f(x, y)
    

In [44]:
my_GD_LRS(Himmelblau,Himmelblau_Grad,x,y)

(array([[-1.06332714],
        [ 5.86606369]]),
 array([[-0.13452826],
        [ 0.86547174]]),
 array([[-164.80300229],
        [-589.48081385]]))

In [107]:
def my_GD_LRS_mom(f: Mapping, df: Mapping, x: np.ndarray, y: np.ndarray, lr: float = 0.01,
          Epoch: int = 100, lre: int = 7, g: float = 0.1, momentum: float = 0.1) -> Tuple [np.ndarray, np.ndarray, np.float32]:
    '''
    Моя простейшая реализация градиентного спуска + Learning Rate Schedule.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка 1
        y0 (np.ndarray): Стартовая точка 2
        lr (float): Скорость обучения. Default=0,01.
        T (int): Количество итераций.
        lre (int): Через сколько эпох мы уменьшим lr. Default = 7.
        g (float): Коэфициент уменьшения lr [0,1). Default = 0.1
    
    Returns:
        Tuple [np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
    
    '''
    n=np.zeros(df(x, y).shape)
    for i in range(Epoch):
        if ((i%lre)==0):
            lr *= g
        n=momentum*n-lr*df(x, y)
        x = x + n[0]
        y = y + n[1]
    return x, y, f(x, y)
    

In [114]:
def my_GD_LRS_nest(f: Mapping, df: Mapping, x0: np.ndarray, y0: np.ndarray, lr: float = 0.01,
          Epoch: int = 100, lre: int = 7, g: float = 0.1, momentum: float = 0.1) -> Tuple [np.ndarray, np.ndarray, np.float32]:
    '''
    Моя простейшая реализация градиентного спуска + Learning Rate Schedule.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка 1
        y0 (np.ndarray): Стартовая точка 2
        lr (float): Скорость обучения. Default=0,01.
        T (int): Количество итераций.
        lre (int): Через сколько эпох мы уменьшим lr. Default = 7.
        g (float): Коэфициент уменьшения lr [0,1). Default = 0.1
    
    Returns:
        Tuple [np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
    
    '''
    x = x0;
    y = y0;
    n=np.zeros(df(x, y).shape)
    for i in range(Epoch):
        if ((i%lre)==0):
            lr *= g
        n=momentum*n-lr*df(x+momentum*n[0], y+momentum*n[1])
        x = x + n[0]
        y = y + n[1]
        
    return x, y, f(x, y)

In [106]:
stochastic_sample(x,y)

(array([-1.29205812]), 0)

In [45]:
#тест GD 1
my_GD_LRS(Himmelblau, Himmelblau_Grad, x, y)
#тест GD+LRS 1
# xlr,ylr,zlr=my_GD_LRS(Himmelblau, Himmelblau_Grad, x, y)

(array([[-1.06332714],
        [ 5.86606369]]),
 array([[-0.13452826],
        [ 0.86547174]]),
 array([[-164.80300229],
        [-589.48081385]]))

In [ ]:
#функция 1
def McCormick(x: np.float32, y: np.float32) -> np.float64:
    '''
    Функция Химмельблау
    
    Args:
        x(np.ndarray): Аргумент 1
        y(np.ndarray): Аргумент 2
        
    Returns:
        np.float64: Результат функции
    '''
    return np.sin(x + y)+np.square(x - y)-1.5*x+2.5*y+1;

#градиент 1
def McCormick_Grad(x: np.ndarray, y: np.ndarray) -> np.float64:
    return np.array([np.cos(x+y)+2*(x-y)-1.5, np.cos(x+y)-2*(x-y)+2.5]);

McCormick(2.5, 3.4), McCormick_Grad(2.5, 3.4)